V tej beležnici modeliramo teme izbranih člankov in shranimo dobljene rezultate.
Interpretacija rezultatov je v beležnici topic_interpretation.

In [ ]:
!pip install pyLDAvis==2.1.2

**Uvažanje knjižnic**

Spremenite pot v vrstici 7 tako, da predstavlja pot do mape, kjer se nahaja datoteka utils s pomožnimi funkcijami

In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora

import sys
utils_dir = '/content/drive/MyDrive/Diploma'
sys.path.insert(0, utils_dir)
import utils as utils


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pyLDAvis, pyLDAvis.gensim as gensimvis
import matplotlib.pyplot as plt

Priprava konstant

In [ ]:
YEAR = 2019

# Po potrebi spremenite pot v naslednji vrstici tako, da pot predstavlja mapo, v kateri so shranjeni model LDA, slovar in korpus
DIR = f'/content/drive/MyDrive/Diploma/lda_models/{YEAR}'

MEDIA_LIST = ['Dnevnik', 'Siol.net Novice', '24ur.com', 'MMC RTV Slovenija', 'Nova24TV', 'Tednik Demokracija', 'PortalPolitikis']
MAINSTREAM_MEDIA_LIST = ['Dnevnik', 'Siol.net Novice', '24ur.com', 'MMC RTV Slovenija']
RIGHT_MEDIA_LIST = ['Nova24TV', 'Tednik Demokracija', 'PortalPolitikis']

Branje in delitev podatkov

In [ ]:
df = pd.read_pickle(f'/content/drive/MyDrive/Diploma/data/data{YEAR}.pkl')

df_mainstream = df.loc[df.media.isin(MAINSTREAM_MEDIA_LIST)]
df_right = df.loc[df.media.isin(RIGHT_MEDIA_LIST)]

In [ ]:
df

Generiranje in shranjevanje slovarja in korpusa

In [ ]:
dictionary, corpus, preprocessed_articles = utils.LDA_data_preparation(df_mainstream, n_gram=2, min_count=15, threshold=100, no_below=10, no_above=0.9, keep_n=100000)

dictionary.save(f'{DIR}/dictionary{YEAR}')
corpora.MmCorpus.serialize(f'{DIR}/corpus{YEAR}', corpus)

Učenje modelov LDA

In [ ]:
# Parametri za učenje LDA. Za informacije o parametrih poglejte https://radimrehurek.com/gensim/models/ldamodel.html
lda_parameters = {
    'chunksize': 50000,
    'passes': 10,
    'iterations': 800,
    'eval_every': None
}

# Parametri za učenje modela LDA in parametri, ki povejo število tem za računanje modela LDA
# Za računanje enega samega modela nastavite limit na start + 1
parameters = {
    'lda_parameters': lda_parameters,
    'limit': 15,
    'start': 10,
    'step': 2
}

filepath = f'{DIR}/lda{YEAR}'
model_list, coherence_values = utils.compute_coherence_values(dictionary, corpus, df_mainstream['preprocessed_body'].to_list(), filepath, **parameters)

best_ix = coherence_values.index(max(coherence_values))
lda_model = model_list[best_ix]
c_v = coherence_values[best_ix]

print(c_v)

Vizualizacija modela LDA z orodje pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
topic_data = gensimvis.prepare(lda_model, corpus, dictionary=lda_model.id2word, sort_topics=False)
pyLDAvis.display(topic_data)

Določanje tem posameznim člankom



In [ ]:
df_mainstream = utils.assign_doc_topics(df_mainstream, lda_model, dictionary)
df_right = utils.assign_doc_topics(df_right, lda_model, dictionary)
df = df_mainstream.append(df_right, ignore_index=True)

In [ ]:
df